In [1]:
!pip install -q ipython-autotime
%load_ext autotime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 25.8 MB/s eta 0:00:00
time: 252 µs (started: 2024-02-19 18:58:24 +00:00)


In [2]:
!pip install -qqq git+https://github.com/huggingface/transformers
!pip install -qqq git+https://github.com/huggingface/datasets
!pip install -qqq git+https://github.com/huggingface/peft
!pip install -qqq git+https://github.com/huggingface/accelerate
!pip install -qqq --upgrade bitsandbytes
!pip install -qqq git+https://github.com/huggingface/trl
!pip install -qqq --upgrade safetensors
!pip install -qqq -U flash-attn
!pip install -qqq evaluate
!pip install -qqq gputil psutil
!pip install -qqq wandb

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 43.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 7.1.0 requires pyarrow<15,>=2, but you have pyarrow 15.0.0 which is incompatible.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 7.1 MB/s eta 0:

In [3]:
import wandb

time: 419 ms (started: 2024-02-19 19:00:54 +00:00)


In [4]:
wandb.init(project="llama2_qa_sft_final", entity="drishtisharma96505", group='packing=False')


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


time: 37.2 s (started: 2024-02-19 19:01:07 +00:00)


In [5]:
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline
)
from peft import LoraConfig
from trl import SFTTrainer

time: 8.06 s (started: 2024-02-19 19:01:45 +00:00)


In [6]:
# Dataset
data_name = "mlabonne/guanaco-llama2-1k"
dataset = load_dataset(data_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

time: 11.5 s (started: 2024-02-19 19:01:53 +00:00)


In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 1000
    })
})

time: 2.39 ms (started: 2024-02-19 19:02:04 +00:00)


In [8]:
from datasets import load_dataset, Dataset, DatasetDict

time: 481 µs (started: 2024-02-19 19:02:04 +00:00)


In [9]:
# Define the seed for reproducibility
seed = 42

# Split ratios for the remaining dataset after extracting the test set
train_split = 820 / (820 + 150)  # Calculate train split ratio from remaining data

# Use DatasetDict's train_test_split to split dataset into test and temp_train (train + validation)
split_dataset = dataset['train'].train_test_split(test_size=30, seed=seed)

# Further split temp_train into actual train and validation sets
split_train = split_dataset['train'].train_test_split(test_size=1-train_split, seed=seed)

# Assemble the final DatasetDict with train, validation, and test splits
dataset = DatasetDict({
    'train': split_train['train'],
    'validation': split_train['test'],
    'test': split_dataset['test']
})

time: 13.7 ms (started: 2024-02-19 19:02:05 +00:00)


In [10]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 820
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 150
    })
    test: Dataset({
        features: ['text'],
        num_rows: 30
    })
})

time: 2.33 ms (started: 2024-02-19 19:02:05 +00:00)


In [11]:
def calculate_text_stats(dataset):
    stats = {}
    for split in ['train', 'validation', 'test']:
        word_lengths = [len(word) for text in dataset[split]['text'] for word in text.split()]
        min_length = min(word_lengths)
        max_length = max(word_lengths)
        avg_length = sum(word_lengths) / len(word_lengths)
        stats[split] = {
            'min_word_length': min_length,
            'max_word_length': max_length,
            'avg_word_length': avg_length
        }
    return stats

time: 1.31 ms (started: 2024-02-19 19:02:05 +00:00)


In [12]:
calculate_text_stats(dataset)

{'train': {'min_word_length': 1,
  'max_word_length': 512,
  'avg_word_length': 5.324315231568705},
 'validation': {'min_word_length': 1,
  'max_word_length': 229,
  'avg_word_length': 5.2476042165788215},
 'test': {'min_word_length': 1,
  'max_word_length': 87,
  'avg_word_length': 5.468633771386065}}

time: 53.2 ms (started: 2024-02-19 19:02:05 +00:00)


In [13]:
# Model and tokenizer names
base_model_name = "NousResearch/Llama-2-7b-chat-hf"


# Tokenizer
llama_tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
llama_tokenizer.pad_token = llama_tokenizer.eos_token

# Quantization Config
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False
)

# Model
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=quant_config,
    attn_implementation="flash_attention_2",
    device_map={"": 0}
)


tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/179 [00:00<?, ?B/s]

time: 56.1 s (started: 2024-02-19 19:02:05 +00:00)


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:411: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:416: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [14]:
# LoRA Config
peft_parameters = LoraConfig(
    lora_alpha=64,
    lora_dropout=0.0,
    r=64,
    bias="none",
    task_type="CAUSAL_LM"
)



time: 931 µs (started: 2024-02-19 19:03:01 +00:00)


In [16]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir= f"llama2-7b-chat-guanaco-llama2-1k-flashatn2-without-packing",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    optim="paged_adamw_32bit",
    save_strategy="epoch",
    evaluation_strategy="epoch",   # Enable evaluation every 'eval_steps'
    logging_strategy='epoch', # log according to log_steps
    learning_rate=2e-4,
    fp16=not torch.cuda.is_bf16_supported(),
    bf16=torch.cuda.is_bf16_supported(),
    lr_scheduler_type="cosine",
    group_by_length=True,
    save_safetensors=True,
    report_to="wandb",
    seed=42
)


time: 2.44 ms (started: 2024-02-19 19:03:19 +00:00)


In [17]:
from trl import SFTTrainer

time: 726 µs (started: 2024-02-19 19:03:20 +00:00)


In [18]:
# Trainer
trainer = SFTTrainer(
    model=base_model,
    train_dataset=dataset["train"],
    eval_dataset = dataset["validation"],
    peft_config=peft_parameters,
    dataset_text_field="text",
    tokenizer=llama_tokenizer,
    args=args,
    packing = False
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:225: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/820 [00:00<?, ? examples/s]

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

time: 921 ms (started: 2024-02-19 19:03:22 +00:00)


/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:294: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:437: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [19]:
# Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA A100-SXM4-40GB. Max memory = 39.564 GB.
5.246 GB of memory reserved.
time: 3.24 ms (started: 2024-02-19 19:03:23 +00:00)


In [20]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
overlay         167G   41G  127G  25% /
tmpfs            64M     0   64M   0% /dev
shm              41G  4.0K   41G   1% /dev/shm
/dev/root       2.0G  1.1G  849M  57% /usr/sbin/docker-init
tmpfs            42G  3.9M   42G   1% /var/colab
/dev/sda1       174G   43G  131G  25% /opt/bin/.nvidia
tmpfs            42G     0   42G   0% /proc/acpi
tmpfs            42G     0   42G   0% /proc/scsi
tmpfs            42G     0   42G   0% /sys/firmware
time: 106 ms (started: 2024-02-19 19:03:24 +00:00)


In [21]:
import time

time: 917 µs (started: 2024-02-19 19:03:25 +00:00)


In [22]:
start_time = time.time()  # Start timer

trainer_stats = trainer.train()

end_time = time.time()  # End timer
training_duration = (end_time - start_time)/60

print(f"Training Time: {training_duration} minutes")

The input hidden states seems to be silently casted in float32, this might be related to the fact you have upcasted embedding or layer norm layers in float32. We will cast back the input in torch.bfloat16.


Epoch,Training Loss,Validation Loss
1,1.313500,1.249047
2,1.117000,1.254800
3,0.959100,1.283430


Training Time: 9.790248815218607 minutes
time: 9min 47s (started: 2024-02-19 19:03:25 +00:00)


In [23]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

586.8049 seconds used for training.
9.78 minutes used for training.
Peak reserved memory = 28.836 GB.
Peak reserved memory for training = 23.59 GB.
Peak reserved memory % of max memory = 72.884 %.
Peak reserved memory for training % of max memory = 59.625 %.
time: 7.96 ms (started: 2024-02-19 19:13:13 +00:00)


In [24]:
from huggingface_hub import notebook_login
notebook_login()

time: 20.3 ms (started: 2024-02-19 19:13:13 +00:00)


In [25]:
trainer.push_to_hub()

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.98k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/DrishtiSharma/llama2-7b-chat-guanaco-llama2-1k-flashatn2-without-packing/commit/0793cc0fea803123e5248c9510ba55ccbdfab821', commit_message='End of training', commit_description='', oid='0793cc0fea803123e5248c9510ba55ccbdfab821', pr_url=None, pr_revision=None, pr_num=None)

time: 15.7 s (started: 2024-02-19 19:14:24 +00:00)


In [26]:
# Generate Text
query = "How do I use the OpenAI API?"
text_gen = pipeline(task="text-generation", model="llama2-7b-chat-guanaco-llama2-1k-flashatn2-without-packing", tokenizer=llama_tokenizer, max_length=200)
output = text_gen(f"<s>[INST] {query} [/INST]")
print(output[0]['generated_text'])

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:411: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:416: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuratio

<s>[INST] How do I use the OpenAI API? [/INST]  The OpenAI API is a RESTful API that allows developers to access the capabilities of the OpenAI models, such as the transformer and the GPT-2, in their own applications. Here are the general steps to use the OpenAI API:

1. Sign up for an OpenAI account: To use the OpenAI API, you need to sign up for an OpenAI account on their website. Once you have an account, you can obtain an API key, which you will use to authenticate your API requests.
2. Choose a programming language: The OpenAI API supports several programming languages, including Python, JavaScript, and Ruby. Choose the language that you are most comfortable with and install the appropriate library or framework to interact with the API.
3. Install the OpenAI library: The OpenAI API provides libraries for several programming languages, including Python, JavaScript,
time: 1min 44s (started: 2024-02-19 19:14:40 +00:00)


In [27]:
# Generate Text
query = "How do I use the OpenAI API?"
text_gen = pipeline(task="text-generation", model="llama2-7b-chat-guanaco-llama2-1k-flashatn2-without-packing", tokenizer=llama_tokenizer, max_length=200)
output = text_gen(f"### Human {query}")
print(output[0]['generated_text'])

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### Human How do I use the OpenAI API?

To use the OpenAI API, you will need to create an account on the OpenAI website and obtain an API key. Once you have an API key, you can use it to make requests to the OpenAI API.

Here is an example of how to use the OpenAI API to generate text:
```
# Import the OpenAI API client
from openai.api import Client

# Set the API endpoint and API key
endpoint = "https://api.openai.com/v1"
api_key = "YOUR_API_KEY"

# Create a client instance
client = Client(endpoint=endpoint, api_key=api_key)

# Generate text using the "text" endpoint
response = client.text("generate", input="Hello, world!")

# Print the generated text
print(response)
```

time: 1min 46s (started: 2024-02-19 19:16:24 +00:00)


In [28]:
# Generate Text
query = "How do I use the OpenAI API?"
text_gen = pipeline(task="text-generation", model="llama2-7b-chat-guanaco-llama2-1k-flashatn2-without-packing", tokenizer=llama_tokenizer, max_length=200)
output = text_gen(f"<s>[INST] {query} [/INST]")
print(output[0]['generated_text'])

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

<s>[INST] How do I use the OpenAI API? [/INST]  The OpenAI API is a RESTful API that allows developers to access the capabilities of the OpenAI models, such as the transformer and the GPT-2, in their own applications. Here are the general steps to use the OpenAI API:

1. Sign up for an OpenAI account: To use the OpenAI API, you need to sign up for an OpenAI account on their website. Once you have an account, you can obtain an API key, which you will use to authenticate your API requests.
2. Choose a programming language: The OpenAI API supports several programming languages, including Python, JavaScript, and Ruby. Choose the language that you are most comfortable with and install the appropriate library or framework to interact with the API.
3. Install the OpenAI library: The OpenAI API provides libraries for several programming languages, including Python, JavaScript,
time: 1min 42s (started: 2024-02-19 19:18:11 +00:00)


In [29]:
# Generate Text
query = "How do I use the OpenAI API?"
text_gen = pipeline(task="text-generation", model="llama2-7b-chat-guanaco-llama2-1k-flashatn2-without-packing", tokenizer=llama_tokenizer, max_length=200)
output = text_gen(f"<s>[INST] {query} [/INST] </s>")
print(output[0]['generated_text'])

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

<s>[INST] How do I use the OpenAI API? [/INST] </s>t is important to note that the OpenAI API is a powerful tool that can be used for a wide range of applications, but it is also a complex and constantly evolving technology. As such, it is important to approach its use with caution and to carefully consider the potential risks and limitations involved.

Here are some general steps that you can follow to use the OpenAI API:

1. Sign up for an OpenAI account: To use the OpenAI API, you will need to sign up for an account on the OpenAI website. This will give you access to the API documentation and sample code.
2. Choose the right API endpoint: The OpenAI API provides a number of different endpoints that you can use depending on the specific task you want to perform. For example, the `text` endpoint can be used to generate text, while the `image`
time: 1min 38s (started: 2024-02-19 19:19:53 +00:00)


In [30]:
# Generate Text
query = "How do I use the OpenAI API?"
text_gen = pipeline(task="text-generation", model="llama2-7b-chat-guanaco-llama2-1k-flashatn2-without-packing", tokenizer=llama_tokenizer, max_length=200)
output = text_gen(f"### Instruction {query}")
print(output[0]['generated_text'])

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### Instruction How do I use the OpenAI API?

To use the OpenAI API, you will need to create an account on the OpenAI website and obtain an API key. Once you have an API key, you can use it to make requests to the OpenAI API.

Here is an example of how to use the OpenAI API to generate text:
```
# Import the OpenAI API client
from openai.api import Client

# Set the API endpoint and API key
endpoint = "https://api.openai.com/v1"
api_key = "YOUR_API_KEY"

# Create a client instance
client = Client(endpoint=endpoint, api_key=api_key)

# Generate text using the "text" endpoint
response = client.text("generate", input="Hello, world!")

# Print the generated text
print(response)
```
time: 1min 44s (started: 2024-02-19 19:21:32 +00:00)


In [31]:
# Generate Text
query = "How do I use the OpenAI API?"
text_gen = pipeline(task="text-generation", model="llama2-7b-chat-guanaco-llama2-1k-flashatn2-without-packing", tokenizer=llama_tokenizer, max_length=200)
output = text_gen(f"### Input {query}")
print(output[0]['generated_text'])

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### Input How do I use the OpenAI API?

The OpenAI API is a RESTful API that allows you to interact with the OpenAI models and services. Here's a step-by-step guide on how to use the OpenAI API:

1. **Sign up for an OpenAI account**: To use the OpenAI API, you need to sign up for an OpenAI account on their website. Once you have an account, you can obtain an API key, which you will use to authenticate your API requests.
2. **Obtain an API key**: After signing up for an OpenAI account, you can obtain an API key by following these steps:
	* Log in to your OpenAI account on the OpenAI website.
	* Click on the "API Keys" tab in the sidebar.
	* Click on the "Create API Key" button.
	* Cho
time: 1min 41s (started: 2024-02-19 19:23:17 +00:00)


In [32]:
print(output)

[{'generated_text': '### Input How do I use the OpenAI API?\n===============================================\n\nThe OpenAI API is a RESTful API that allows you to interact with the OpenAI models and services. Here\'s a step-by-step guide on how to use the OpenAI API:\n\n1. **Sign up for an OpenAI account**: To use the OpenAI API, you need to sign up for an OpenAI account on their website. Once you have an account, you can obtain an API key, which you will use to authenticate your API requests.\n2. **Obtain an API key**: After signing up for an OpenAI account, you can obtain an API key by following these steps:\n\t* Log in to your OpenAI account on the OpenAI website.\n\t* Click on the "API Keys" tab in the sidebar.\n\t* Click on the "Create API Key" button.\n\t* Cho'}]
time: 840 µs (started: 2024-02-19 19:24:58 +00:00)
